In [1]:
# === Imports ===
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
import os
from dotenv import load_dotenv

# === Load API Key ===
load_dotenv()
api_key = os.getenv("API_KEY")

# === Initialize Gemini ===
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=api_key,
    temperature=0.3
)

# === Embeddings ===
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=api_key
)

# === Load CSV ===
csv_path = "employees.csv"   # 👈 replace with your CSV file
loader = CSVLoader(file_path=csv_path, encoding="utf-8")
data = loader.load()

# === Split CSV rows into chunks ===
splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(data)

# === Create Vector Store ===
vectorstore = FAISS.from_documents(docs, embeddings)

# === Conversational Retrieval Chain ===
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    memory=memory,
    return_source_documents=False
)

# === Chat with CSV ===
print("CSV Q&A Bot (Type 'quit' to exit)")
while True:
    query = input("\nYou: ")
    if query.lower() == "quit":
        print("Goodbye!")
        break
    
    result = qa_chain.invoke({"question": query})
    print("Gemini:", result["answer"])


C:\Users\lokes\AppData\Local\Temp\ipykernel_4520\2719684592.py:41: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key="answer")


CSV Q&A Bot (Type 'quit' to exit)
Gemini: John Seo, Ki Gee, Neena Kochhar, John Chen
Gemini: Steven King had the highest salary at $24000.
Goodbye!
